In [1594]:
import pandas as pd 
import numpy as np

In [1595]:
#Importing a CSV File using read_csv() method

sales_2017        = pd.read_csv(r"C:\Users\nares\Desktop\SQL & Pandas\sales2017_uncleaned.csv")
sales_2018        = pd.read_csv(r"C:\Users\nares\Desktop\SQL & Pandas\sales2018.csv")
sales_2019        = pd.read_csv(r"C:\Users\nares\Desktop\SQL & Pandas\sales2019.csv")
product_details   = pd.read_csv(r"C:\Users\nares\Desktop\SQL & Pandas\product_details.csv")
store_cities      = pd.read_csv(r"C:\Users\nares\Desktop\SQL & Pandas\store_cities.csv")

In [1596]:
#Concatenating all the sales Files 

sales_2017_2019 = pd.concat((sales_2017,sales_2018,sales_2019))
sales_2017_2019.shape

(88158, 17)

In [1597]:
                            '''TRANSFORMING THE DATA FOR COMPATIBILITY AND CONSISTENCY'''

#Dropping the Unwanted and Redundant Columns:
sales_2017_2019.drop(['column3','order_date_2','promo_type_1','promo_bin_1','promo_type_2',
                      'promo_bin_2','promo_discount_2','delivery_date_format2'],axis=1,inplace=True)

sales_2017_2019.shape

(88158, 9)

In [1598]:
#Dropping the Duplicates
sales_2017_2019.drop_duplicates(inplace=True)
sales_2017_2019.shape

(88152, 9)

In [1599]:
#Count of null values in each and every column 
sales_2017_2019.isna().sum()

order_id                    1
product_id                  3
store_id                    3
order_date                  3
sales                    8133
revenue                  8133
stock                    8133
price                    2117
delivery_date_format1       3
dtype: int64

In [1600]:
#Renaming the Columns :
sales_2017_2019 = sales_2017_2019.rename(columns ={'delivery_date_format1':'delivery_date'})

In [1601]:
#Removing the Null values :
sales_2017_2019.dropna(subset= ['order_id','product_id','store_id','order_date','delivery_date'],inplace=True)

In [1602]:
#Filling Null Values with the Mean :
sales = sales_2017_2019 
sales['revenue']   = round(sales['revenue'].fillna(sales.groupby(['product_id'])['revenue'].transform('mean')),2)
sales['price']     = round(sales['price'].fillna(sales.groupby(['product_id'])['price'].transform('mean')),2)
sales['stock']     = round(sales['stock'].fillna(sales.groupby(['store_id'])['stock'].transform('mean')),2) 

#Dropping the Null values after filling with the Mean:
sales.dropna(subset=['stock','price'],inplace =True)
sales.isna().sum()

order_id            0
product_id          0
store_id            0
order_date          0
sales            8104
revenue           342
stock               0
price               0
delivery_date       0
dtype: int64

In [1603]:
#Trimming the Columns using Strip() And Filling the null values using Interpolate Method:
sales['sales'] = sales['sales'].str.strip('sales').interpolate(method='linear',direction='forward').fillna(0)

In [1604]:
#Updating the Datatype of Columns:
sales['sales']          = sales['sales'].astype('float').astype('int')
sales['order_id']       = sales['order_id'].astype('int')
sales['stock']          = sales['stock'].astype('int') 
sales['delivery_date']  = pd.to_datetime(sales['delivery_date'])
sales['order_date']     = pd.to_datetime(sales['order_date'])

In [1605]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87993 entries, 1 to 47238
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       87993 non-null  int32         
 1   product_id     87993 non-null  object        
 2   store_id       87993 non-null  object        
 3   order_date     87993 non-null  datetime64[ns]
 4   sales          87993 non-null  int32         
 5   revenue        87651 non-null  float64       
 6   stock          87993 non-null  int32         
 7   price          87993 non-null  float64       
 8   delivery_date  87993 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int32(3), object(2)
memory usage: 5.7+ MB


In [1606]:
#Returning First 5 Rows using head() :
sales.head()

,order_id,product_id,store_id,order_date,sales,revenue,stock,price,delivery_date
1,1,P0258,S0008,2017-01-02,0,0.00,5,6.50,2017-01-05
4,2,P0348,S0110,2017-01-02,0,0.00,9,2.10,2017-01-04
6,3,P0219,S0026,2017-01-02,5,33.56,3,7.25,2017-01-05
10,4,P0218,S0051,2017-01-02,0,0.00,1,59.90,2017-01-04
11,5,P0660,S0056,2017-01-02,1,4.58,5,4.95,2017-01-04


In [1607]:
#Statiscal information of Dataframe sales
sales.describe()

,order_id,sales,revenue,stock,price
count,87993.000000,87993.000000,87651.000000,87993.000000,87993.000000
mean,44034.816258,0.101542,2.166738,15.974055,16.741538
std,25444.515020,1.137781,17.120361,46.612896,34.933354
min,1.000000,0.000000,0.000000,0.000000,0.010000
25%,22000.000000,0.000000,0.000000,4.000000,3.500000
50%,43997.000000,0.000000,0.000000,9.000000,8.500000
75%,66073.000000,0.000000,0.000000,17.000000,17.990000
max,88149.000000,178.000000,2497.960000,3993.000000,1549.000000


In [1608]:
#Merging the Dataframes
sales_product_details = pd.merge(sales,product_details,how='inner',on='product_id')
sales                 = pd.merge(sales_product_details,store_cities,how='inner',on='store_id')
sales.head()

,order_id,product_id,store_id,order_date,sales,revenue,stock,price,delivery_date,product,type,category,sub_category,state,city
0,1,P0258,S0008,2017-01-02,0,0.0,5,6.50,2017-01-05,Sunfeast,Shower Gel & Body Wash,Beauty & Hygiene,Bath & Hand Wash,Oklahoma,Oklahoma City
1,1625,P0258,S0008,2017-02-08,0,0.0,10,6.50,2017-02-10,Sunfeast,Shower Gel & Body Wash,Beauty & Hygiene,Bath & Hand Wash,Oklahoma,Oklahoma City
2,16788,P0258,S0008,2017-12-06,0,0.0,3,7.10,2017-12-08,Sunfeast,Shower Gel & Body Wash,Beauty & Hygiene,Bath & Hand Wash,Oklahoma,Oklahoma City
3,7141,P0348,S0008,2017-06-06,0,0.0,14,2.15,2017-06-08,Kohinoor,Herbs & Seasoning,"Foodgrains, Oil & Masala",Masalas & Spices,Oklahoma,Oklahoma City
4,9577,P0348,S0008,2017-07-26,0,0.0,11,2.30,2017-07-29,Kohinoor,Herbs & Seasoning,"Foodgrains, Oil & Masala",Masalas & Spices,Oklahoma,Oklahoma City


In [1609]:
                            '''GAINING INSIGHTS FROM STORES DATA TO BOOST BUSINESS PERFORMANCE'''
# No of Unique products/stores 
def Unique_count(Column):
  Count = sales[Column].nunique()
  return Count 

In [1610]:
# product_count 
Unique_count('product_id')

603

In [1611]:
#store_count 
Unique_count('store_id')

100

In [1612]:
# Which Product/category is Performing better and poor ? 
def Total_revenue(column,boolean_value):
    query_1 = round(
               sales.groupby([column]).agg({'revenue':'sum'})
               .reset_index().sort_values(by=['revenue'],ascending=[boolean_value])
               .head(1),2
             )
    return query_1 

In [1613]:
#Best Product based on revenue 
Total_revenue('product',False)

,product,revenue
62,Fresho,27639.4


In [1614]:
#Product which is not performing up to the mark based on revenue 
Total_revenue('product',True)

,product,revenue
152,Sakthi,0.0


In [1615]:
#Category with highest_avg_revenue 
def Avg_revenue(column,boolean_value):
    query_1 = round(
               sales.groupby([column]).agg({'revenue':'sum'})
               .reset_index().sort_values(by=['revenue'],ascending=[boolean_value])
               .head(1),2
             )
    return query_1 

Avg_revenue('category',False)

,category,revenue
9,Snacks & Branded Foods,43805.64


In [1616]:
#Category with least_avg_revenue 
Avg_revenue('category',True)

,category,revenue
0,Baby Care,175.36


In [1617]:
# State With highest Avg_revenue 
Avg_revenue('state',False)  

,state,revenue
30,Texas,21033.18


In [1618]:
#state with least Avg_revenue 
Avg_revenue('state',True)

,state,revenue
11,Idaho,84.42


In [1619]:
#Most No of times ordered Products 
(  sales.groupby(['product']).agg({'sales':'sum'})
  .reset_index().rename(columns={'sales':'No of times ordered'})
  .sort_values(by=['No of times ordered'],ascending=[False]).head()
 )  

,product,No of times ordered
62,Fresho,931
159,Sln,657
120,Nestle,462
25,Britannia,428
187,bb Popular,229


In [1620]:
#Top 2 Performing Products in each and every Category 
query_2 = ( sales.groupby(['category','product']).agg({'revenue':'sum'})
            .reset_index().sort_values(by=['revenue'],ascending=[False])
            .groupby(['category']).head(2)
            .sort_values(by=['category','revenue'],ascending=[True,False])
          )
query_2

,category,product,revenue
0,Baby Care,Bisleri,175.36
1,Baby Care,Patanjali,0.00
4,"Bakery, Cakes & Dairy",Amul,2254.67
15,"Bakery, Cakes & Dairy",English Oven,879.67
58,Beauty & Hygiene,bb Royal,4483.76
34,Beauty & Hygiene,Britannia,1183.28
75,Beverages,bb Royal,3218.34
65,Beverages,Fresho,1403.22
86,Cleaning & Household,Fresho,1254.05
81,Cleaning & Household,B Vishal,1194.20


In [1621]:
#Stock Left in Each city 
stock_left = sales.groupby(['state']).agg({'stock':'sum'}).reset_index().sort_values(by=['stock'],ascending=[False])
stock_left

,state,stock
30,Texas,160447
3,California,108624
8,Florida,79848
25,Ohio,62081
20,Minnesota,41499
14,Iowa,39204
32,Virginia,37560
4,Colorado,37427
0,Alabama,36251
16,Kentucky,26623


In [1622]:
#Comparision of Revenue Over the Years/Months - Sales Growth 
def Date_wise_Revenue(date_part):
    query_3 = round(
                     sales.groupby([date_part]).agg({'revenue':'sum'})
                     .reset_index().astype('int')
                     .rename(columns={'order_date':'year'})
                     .sort_values(by=['revenue'],ascending=[False])
                    )
    return query_3 

In [1623]:
#Year wise Avg_revenue  
Date_wise_Revenue(sales['order_date'].dt.year)

,year,revenue
2,2019,74650
1,2018,28227
0,2017,19785


In [1624]:
#Month wise Avg_revenue 
sales['Year'] = sales['order_date'].dt.year 
sales['month'] = sales['order_date'].dt.month
def Month_wise_Revenue(Y,M):
    query_3 = round(
                     sales.groupby([Y,M]).agg({'revenue':'sum'})
                     .reset_index().astype('int')
                     .sort_values(by=['Year','revenue'],ascending=[True,False])
                    )
    return query_3 
#Mapping month Numbers with Month names
Month_wise          = Month_wise_Revenue(sales['Year'],sales['month'])
Month_names         = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'} 
Month_wise['month'] = Month_wise['month'].map(Month_names)
Month_wise

,Year,month,revenue
9,2017,Oct,1914
11,2017,Dec,1834
7,2017,Aug,1742
2,2017,Mar,1735
4,2017,May,1732
5,2017,Jun,1698
6,2017,Jul,1673
8,2017,Sep,1672
3,2017,Apr,1561
10,2017,Nov,1541
